In [8]:
# !pip install statsmodels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import LinearRegression

from statsmodels.regression.quantile_regression import QuantReg
import statsmodels.formula.api as smf
from scipy.stats import norm

In [2]:
url = 'https://raw.githubusercontent.com/mnuzen/156b-learning-support/master/data/us/covid/deaths.csv'
df = pd.read_csv(url, header=0)

In [3]:
cum_cases = df.iloc[:, 4:]
countyFIPS = df.iloc[:, 0].to_numpy()
cum_cases = cum_cases.to_numpy()
print(cum_cases.shape)

(3195, 109)


In [4]:
all_zeros = [0 for i in range(91)]
PREDS = []
reg = LinearRegression()
for i in range(len(cum_cases)):
    if cum_cases[i, -1] == 0: # no training
        PREDS.append(all_zeros)
    else:
        y = cum_cases[i]
        j = 0
        while y[j] == 0:
            j+=1
        y = y[j:]
        x = np.array(range(j, j+len(y))).reshape((len(y), 1))
        x_pred = np.array(range(70, 161)).reshape((91,1))
        reg.fit(x, y)
        pred = reg.predict(x_pred)
        PREDS.append(pred)

In [27]:
PREDS = np.array(PREDS)
print(PREDS.shape)
DAILY_PRED = np.zeros((3195,91))
DAILY_PRED[:, 0] = np.subtract(PREDS[:,0], cum_cases[:, 69])
for i in range(1, len(DAILY_PRED[0])):
    DAILY_PRED[:, i] = np.subtract(PREDS[:,i], PREDS[:, i-1])

(3195, 91)


In [28]:
FINAL_PRED = []
dates = np.loadtxt('dates.txt', dtype=np.str)
# assume normal distribution
for county in range(len(DAILY_PRED)):
    for date in range(len(DAILY_PRED[0])):
        mean = DAILY_PRED[county, date]
        std = max(1, mean)**(1/2)
        heading = dates[date] + '-' + str(countyFIPS[county])
        quantiles = np.linspace(norm.ppf(0.1, mean, std), norm.ppf(0.9, mean, std), 9)
        quantiles = quantiles.clip(0).tolist()
        quantiles.insert(0, heading)
        FINAL_PRED.append(quantiles)

In [31]:
FINAL_PRED = np.array(FINAL_PRED)
print(FINAL_PRED.shape)
print(FINAL_PRED[64813])

(290745, 10)
['2020-04-22-0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.32038789138615'
 '0.6407757827723002' '0.9611636741584504' '1.2815515655446004']


In [32]:
df = pd.DataFrame(data={"id": FINAL_PRED[:, 0], 
                        "10": FINAL_PRED[:, 1], 
                        "20": FINAL_PRED[:, 2], 
                        "30": FINAL_PRED[:, 3], 
                        "40": FINAL_PRED[:, 4], 
                        "50": FINAL_PRED[:, 5], 
                        "60": FINAL_PRED[:, 6], 
                        "70": FINAL_PRED[:, 7], 
                        "80": FINAL_PRED[:, 8], 
                        "90": FINAL_PRED[:, 9]})
df.to_csv("./submission_linreg_daily.csv", sep=',',index=False)

In [35]:
url = 'https://raw.githubusercontent.com/mnuzen/156b-learning-support/master/sample_submission.csv'
df = pd.read_csv(url, header=0)
df.to_csv("./sample_submission.csv", sep=',',index=False)